In [1]:
import pandas as pd
import string
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
print(string.punctuation)
print(stop_words)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only

In [2]:
df = pd.read_csv('/work/TextClassification/TextClassification/dataset.csv')

In [3]:
df.head(5)

,category1,category2,category3,body
0,Anonymity Networks,NaN,NaN,\n\t\t\t\n\t\t\t\t\nI just saw a post in TOR s...
1,Anonymity Networks,Cryptography,WriteUp,\n\t\t\t\n\t\t\t\t\nOnionFarm was one of the c...
2,AndroJava,Programming,RAT,\n\t\t\t\n\t\t\t\t\nI was researching about An...
3,IoT,Personal,NaN,\n\t\t\t\n\t\t\t\t\nI’ve previously installed ...
4,AndroJava,Media,Tutorial,\n\t\t\t\n\t\t\t\t\nI previously created a “Hi...


In [4]:
def PreProcess(df):
    tokens = []
    for text in df.body:
        words = word_tokenize(RemovePuncts(text.lower()))
        filtered_tokens = [word for word in words if word not in stop_words and not word.isdigit()]
        tokens.append(" ".join(filtered_tokens))
    return tokens

def RemovePuncts(s):
    return "".join([ch for ch in s if ch not in string.punctuation and ch not in ['”',"’",'“',"."]])

In [5]:
tokens = PreProcess(df)
df.pop('body')
df['abstract'] = tokens

In [6]:
df.head(5)

,category1,category2,category3,abstract
0,Anonymity Networks,NaN,NaN,saw post tor stack exchange titled recommended...
1,Anonymity Networks,Cryptography,WriteUp,onionfarm one challenges uutctf held april urm...
2,AndroJava,Programming,RAT,researching android malware features compare i...
3,IoT,Personal,NaN,ive previously installed home assistant ha hom...
4,AndroJava,Media,Tutorial,previously created hiddenlivecamera library an...


In [7]:
df.count()

category1    55
category2    47
category3    28
abstract     55
dtype: int64

In [8]:
df['category1'].value_counts()

Academic              13
Anonymity Networks    12
AndroJava             10
Personal               6
Media                  4
IoT                    3
Conference             2
Cryptography           2
Cryptocurrency         1
Library                1
Tutorial               1
Name: category1, dtype: int64

In [9]:
df.groupby('category1')['category2'].value_counts()

category1           category2         
Academic            AndroJava             5
                    Tutorial              2
                    University            2
                    Personal              1
                    Programming           1
AndroJava           Anonymity Networks    2
                    Programming           2
                    WriteUp               2
                    Media                 1
                    Personal              1
                    University            1
Anonymity Networks  Cryptography          3
                    Personal              2
                    Conference            1
                    Programming           1
                    Server                1
                    Tutorial              1
                    Urmia                 1
Conference          Cryptography          1
                    University            1
Cryptocurrency      Cryptography          1
Cryptography        Personal         

In [10]:
df.groupby('category1')['category3'].value_counts()

category1           category3    
Academic            Programming      3
                    Search Engine    1
                    Tutorial         1
                    WriteUp          1
AndroJava           Programming      1
                    RAT              1
                    Tutorial         1
                    WriteUp          1
Anonymity Networks  Tutorial         5
                    Search Engine    1
                    WriteUp          1
Conference          Tutorial         1
                    Urmia            1
Cryptocurrency      University       1
Cryptography        Tutorial         1
IoT                 RaspberryPi      1
Library             Programming      1
Media               Personal         2
                    Tutorial         1
Personal            Tutorial         1
                    WriteUp          1
Name: category3, dtype: int64

In [11]:
df.groupby(['category1','category2'])['category3'].value_counts()

category1           category2           category3    
Academic            AndroJava           Programming      3
                                        Search Engine    1
                    Personal            WriteUp          1
                    Programming         Tutorial         1
AndroJava           Anonymity Networks  Programming      1
                    Media               Tutorial         1
                    Personal            WriteUp          1
                    Programming         RAT              1
Anonymity Networks  Cryptography        Tutorial         2
                                        WriteUp          1
                    Personal            Tutorial         2
                    Programming         Search Engine    1
                    Server              Tutorial         1
Conference          Cryptography        Tutorial         1
                    University          Urmia            1
Cryptocurrency      Cryptography        University       1
Cr

In [12]:
# Anonymity Networks : 0
# AndroJava          : 1
# Iot                : 2
# Cryptography       : 3
# Cyber Security     : 4

df['category1'][0] = 0
df['category1'][1] = 0
df['category1'][2] = 1
df['category1'][3] = 2
df['category1'][4] = 1
df['category1'][5] = 0
df['category1'][6] = 0
df['category1'][7] = 0
df['category1'][8] = 1
df['category1'][9] = 1
df = df.drop(10) # media
df['category1'][11] = 4
df['category1'][12] = 4
df = df.drop(13) # personal
df = df.drop(14) # academic
df = df.drop(15) # personal
df = df.drop(16) # personal
df['category1'][17] = 1
df = df.drop(18) # academic
df = df.drop(19) # personal
df['category1'][20] = 2
df['category1'][21] = 3
df['category1'][22] = 4
df['category1'][23] = 2
df['category1'][24] = 3
df['category1'][25] = 1
df = df.drop(26) # academic
df['category1'][27] = 1
df['category1'][28] = 2
df['category1'][29] = 0
df['category1'][30] = 3
df['category1'][31] = 0
df['category1'][32] = 0
df['category1'][33] = 0
df['category1'][34] = 0
df['category1'][35] = 0
df = df.drop(36) # personal
df['category1'][37] = 1
df['category1'][38] = 1
df = df.drop(39) # media
df = df.drop(40) # academic
df = df.drop(41) # academic
df['category1'][42] = 1
df = df.drop(43) # Library
df = df.drop(44) # academic
df = df.drop(45) # academic
df['category1'][46] = 1
df['category1'][47] = 0
df = df.drop(48) # academic
df['category1'][49] = 0
df['category1'][50] = 3
df['category1'][51] = 1
df['category1'][52] = 1
df = df.drop(53) # Conference
df = df.drop(54) # media


In [13]:
df.reset_index(inplace=True,drop=True)
df['category1'].value_counts()

1    13
0    13
3     4
2     4
4     3
Name: category1, dtype: int64

In [14]:
df

,category1,category2,category3,abstract
0,0,NaN,NaN,saw post tor stack exchange titled recommended...
1,0,Cryptography,WriteUp,onionfarm one challenges uutctf held april urm...
2,1,Programming,RAT,researching android malware features compare i...
3,2,Personal,NaN,ive previously installed home assistant ha hom...
4,1,Media,Tutorial,previously created hiddenlivecamera library an...
5,0,Tutorial,NaN,previously ive described run hidden mail servi...
6,0,NaN,NaN,post want talk compiling using tor android pro...
7,0,Cryptography,Tutorial,post talk little onionbalance load balancer to...
8,1,Anonymity Networks,NaN,previous post ive talked compiling tor source ...
9,1,NaN,NaN,post im going talk new library using nmap andr...


In [15]:
X = df['abstract']
Y = df['category1']

# vocabulary
allwords = []
for row in X:
    allwords.append(re.split("\s+",row))

vocab = []
for listofwords in allwords:
    for word in set(listofwords):
        vocab.append(word)


train_x, test_x, train_y, test_y = train_test_split(X,Y,test_size= 0.1,random_state=1319)
train_y = train_y.astype('int')
test_y  = test_y.astype('int')

In [16]:
def bow(dataframe):
    return pd.DataFrame( [ [row.count(word) for word in vocab] for row in dataframe ], columns=vocab)

In [17]:
train_x_voc = bow(train_x)

test_x_voc = bow(test_x)

train_x_voc.head(5)

,know,discussed,privacy,use,extra,add,packets,alongside,stack,vpn,...,inside,two,decompressed,compressed,techniques,piece,write,activity,numpad,oncreate
0,0,0,0,2,0,3,0,0,1,0,...,0,0,0,0,0,0,0,2,0,2
1,1,0,0,9,0,1,0,0,0,0,...,1,3,0,0,0,0,3,0,0,0
2,1,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
3,0,0,0,2,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,4,0,0,0,0,2,0,...,0,1,0,0,0,0,0,0,0,0


In [18]:
mnb = MultinomialNB()
mnb.fit(train_x_voc,train_y)

MultinomialNB()

In [19]:
mnb.score(test_x_voc,test_y)

0.75

In [20]:
text = input("Enter your text").lower()
text_df = pd.DataFrame([text],columns=["body"])
text_df['abstract'] = PreProcess(text_df)
text_df.pop('body')
mnb.predict(bow(text_df))